In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf
import folium
#import bokeh

In [2]:
#source: http://projects.knmi.nl/klimatologie/daggegevens/selectie.cgi
file_name = r'D:\git\pandas-bokeh\data\KNMI_20161227.txt'

In [3]:
def load_weather_data(file_name,station_numbers):
    
    weather_header_list = []
    weather_dict = {}
    spatial_header_list = []
    spatial_dict = {}
    
    def clear_whitespace(element, characters_to_remove):
        characters_in_element = [character for character in list(element) if character not in characters_to_remove]
        cleaned_element = ''.join(characters_in_element)
        return cleaned_element
    
    def process_headers(header_line, header_list, header_dict, separator, unique_id = 'False'):
        selected_header_line = header_line
        split_header_list = selected_header_line.split(separator)
        #Determine headers in the main file (operations in order: remove hashtag, append main body, remove trailing line break)
        uncleaned_header_list = [split_header_list[0][2:]]\
                            + split_header_list[1:-1] \
                            + [split_header_list[-1][:-1]]
                
        for element in uncleaned_header_list:
            cleaned_header = clear_whitespace(element, (' '))
            #Ensure that only valid headers are added (empty = not appended)
            if len(cleaned_header) > 0:
                header_list.append(cleaned_header)
                header_dict[cleaned_header] = []
        if unique_id == True:
            header_list.append('ID')
            header_dict['ID'] = []
            
    
    with open(file_name) as data:
        loaded_data = data.readlines()
        
        '''Process weather data header'''
        process_headers(loaded_data[97], weather_header_list, weather_dict, ',', unique_id = True)
        
        '''Process main weather data'''
        uncleaned_weather_data = [line for line in loaded_data[100:] if line[2:5] in station_numbers]#stationNumber]
        weather_data = [line.split(',') for line in uncleaned_weather_data[:]]        
        
        for linenumber, lines in enumerate(weather_data):
            for element_number, elements in enumerate(lines):
                cleaned_element = clear_whitespace(elements,(' ', '\n'))
                #Error handling required to prevent unexpected EOF while parsing - no known alternatives         
                try:
                    evaluated_value = eval(cleaned_element)
                    weather_dict[weather_header_list[element_number]].append(evaluated_value)                    
                except:
                    weather_dict[weather_header_list[element_number]].append(cleaned_element)
            weather_dict['ID'].append(linenumber)
        
        '''Process spatail data headers'''   
        process_headers(loaded_data[4], spatial_header_list, spatial_dict, ' ')
        
        '''Process main spatial data'''
        uncleaned_spatial_data = loaded_data[5:55]
        #Splitting spatial data lines
        split_spatial_data = [line.split() for line in uncleaned_spatial_data[:]]    
        spatial_data = list(map(lambda values: values[1:], split_spatial_data))
        #Remove trailing colon after first element
        for lines in spatial_data:
            lines[0] = lines[0][:-1]
            while len(lines) > len(spatial_header_list):
                lines[len(spatial_header_list)-1] = str(lines[len(spatial_header_list)-1]) + ' ' + str(lines[len(spatial_header_list)])
                del lines[len(spatial_header_list)]
            for element_number, element in enumerate(lines):
                try:
                    spatial_dict[spatial_header_list[element_number]].append(eval(element))
                except:
                    spatial_dict[spatial_header_list[element_number]].append(element)
        
        '''Combine header files'''
        headers = weather_header_list[:] + spatial_header_list[:]
            
    return weather_dict, spatial_dict, headers

In [5]:
perform_SQL = lambda q: sqldf(q, globals())

In [6]:
#Joining the two datasets together
def join_dataframes(weather_dict, spatial_dict):
    weather_DF = pd.DataFrame(weather_dict)[1:].apply(pd.to_numeric)
    spatial_DF = pd.DataFrame(spatial_dict)

    #Runtime is about 5mins
    sql = """
            SELECT * FROM weather_DF
            JOIN spatial_DF ON spatial_DF.stn = weather_DF.stn;
          """

    weather_and_spatial_data_DF = perform_SQL(sql)
    return weather_and_spatial_data_DF

In [6]:
def get_unique_stations(file_name):
    with open(file_name) as data:
        loaded_data = data.readlines()
        unique_numbers = sorted(list(set(list([line[2:5] for line in loaded_data[100:]]))))
    return unique_numbers
unique_station_numbers = get_unique_stations(file_name)
unique_station_numbers = ['225']

In [15]:
#Runtime for an avg selection takes about 5min per marker
#Gathers data using SQL queries which is then added to the markers

weather_map = folium.Map(location=[52.092560, 5.109378],zoom_start = 8)

for number in unique_station_numbers:
    loaded_data = load_weather_data(file_name, number)

    weather_DF = pd.DataFrame(loaded_data[0])[1:].apply(pd.to_numeric)
    spatial_DF = pd.DataFrame(loaded_data[1])    
    
    #Get weather data
    sql = """
            SELECT STN, MAX(FHX) FROM weather_DF
            WHERE STN = {unique_station_numbers}
          """.format(unique_station_numbers = number)
    max_wind_speed_list = perform_SQL(sql).to_csv(None, header=False, index=False).split('\n')[:-1]
    max_wind_speed_list = max_wind_speed_list[0].split(',')
    
    #Get dates with fastest decreasing temperatures
    sql = """
            SELECT day1.yyyymmdd, day1.ID, day1.TG, IFNULL(day1.TG - day2.TG, 0) AS difference
            FROM weather_DF as day1
                JOIN weather_DF day2 ON day1.STN = day2.STN AND day2.ID - day1.ID == 1
            ORDER BY difference DESC
          """
    biggest_tempdif_list = perform_SQL(sql).to_csv(None, header=False, index=False).split('\n')[:-1]
    biggest_tempdif_list = biggest_tempdif_list[0].split(',')

    try:
        biggest_temperature_formatted = float(biggest_tempdif_list[2])/10
    except:
        biggest_temperature_formatted = "no data"
    temp_dif_date = biggest_tempdif_list[0]
    
    #Get spatial data
    sql = """
            SELECT "LAT(north)", "LON(east)" FROM spatial_DF
            WHERE STN = {unique_station_numbers}
          """.format(unique_station_numbers = number)
    spatial_data_list = perform_SQL(sql).to_csv(None, header=False, index=False).split('\n')[:-1]
    spatial_data_list = spatial_data_list[0].split(',')

    #Create Folium marker
    folium.Marker([spatial_data_list[0], spatial_data_list[1]],\
        popup=folium.Popup(folium.element.IFrame(html=\
        '''
            <b>Station:</b>                              {stn} <br>
            <b>Maximum wind speed:</b>                   {fg} <br>
            <b>Maximum temperature drop in two days:</b> {temp} C <br>
            <b>Date of max temp dif:</b>                 {date}
        '''.format(stn = max_wind_speed_list[0],\
                   fg = max_wind_speed_list[1],\
                   temp = biggest_temperature_formatted,\
                   date = temp_dif_date),\
        width=350, height=120),\
        max_width=350)).add_to(weather_map)
    weather_map.save('weather_map.html')
    
weather_map.save('weather_map.html')
weather_map